importing the stuff and loading the data

CCLE expression data is quantified from RNAseq files using the GTEx pipelines. A detailed description of the pipelines and tool versions can be found here: https://github.com/broadinstitute/ccle_processing#rnaseq. We provide a subset of the data files outputted from this pipeline available on FireCloud. These are aligned to hg38.

### CCLE_expression.csv
Pipeline: Expression

Gene expression TPM values of the protein coding genes for DepMap cell lines. Values are inferred from RNA-seq data using the RSEM tool and are reported after log2 transformation, using a pseudo-count of 1; log2(TPM+1).

Additional RNA-seq-based expression measurements are available for download as part of the full DepMap Data Release

More information on the DepMap Omics processing pipeline is available at <https://github.com/broadinstitute/depmap_omics>.

### sample_info.csv

Metadata for all of DepMap’s cancer models/cell lines. A full description of each column is available in the DepMap Release README file.

Columns:

- DepMap_ID: Static primary key assigned by DepMap to each cell line

- cell_line_name: Original cell line name, including punctuation

- stripped_cell_line_name: Cell line name with alphanumeric characters only

- CCLE_Name: Previous naming system that used the stripped cell line name followed
by the lineage; no longer assigned to new cell lines

- alias: Additional cell line identifiers (not a comprehensive list)

- COSMICID: Cell line ID used in Cosmic cancer database

- sex: Sex of tissue donor if known

- source: Source of cell line vial used by DepMap

- RRID: Cellosaurus research resource identifier

- WTSI_Master_Cell_ID: ID of corresponding record in Sanger Drug dataset

- sample_collection_site: Tissue collection site

- primary_or_metastasis: Indicates whether tissue sample is from primary or metastatic
site

- primary_disease: General cancer lineage category

- Subtype: Subtype of disease; specific disease name

- age: If known, age of tissue donor at time of sample collection

- Sanger_Model_ID: Sanger Institute Cell Model Passport ID

- depmap_public_comments: Further information about the cell line   

- lineage, lineage_subtype, lineage_sub_subtype, lineage_molecular_subtype: Cancer
type classifications in a standardized form

- default_growth_pattern: Typical growth pattern of the cell line

- model_manipulation: Cell line modifications including drug resistance and gene knockout

- model_manipulation_details: Additional information about the model manipulation

- patient_id: Identifier indicating which cell lines come from the same patient

- parent_depmap_id: If known, DepMap ID of parental cell line

- Cellosaurus_NCIt_disease: From Cellosaurus, NCI thesaurus disease term

- Cellosaurus_NCIt_id:  From Cellosaurus, NCI thesaurus code

- Cellosaurus_issues:  From Cellosaurus, documented issues with cell line'

In [1]:
import numpy as np
import pandas as pd
import pickle
#from torchvision  import transforms
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
#load the data into pandas dataframes
with open('data/CCLE_expression.csv') as f:
    genes=pd.read_csv(f)
with open('data/sample_info.csv') as g:
    meta=pd.read_csv(g)

In [3]:
#name the ID column
genes.rename(columns={genes.columns[0]:'id'},inplace=True)
meta.rename(columns={meta.columns[0]:'id'},inplace=True)


In [4]:
#remove features/genes with less than 0.01 standard deviation
#these are housekeeping genes that won't contribute to classification
st = (genes.std()<0.1)
cols_drop = [n for n,c in zip(st.index,st) if c]
gs = genes.drop(cols_drop, axis=1)

C:\Users\boop\AppData\Local\Temp\ipykernel_13456\441055720.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  st = (genes.std()<0.1)


In [21]:
#checking how sparse the matrix is
sparsity = 1.0 - ( np.count_nonzero(genes) / float(genes.size) )
sparsity

0.16965302322951725

In [22]:
#checking which genes have mostly zero expression in collected samples
per_row = (genes == 0).sum(axis=1)
per_col = (genes == 0).sum(axis=0)
(per_col > genes.shape[0]*0.75).sum()

1701

In [5]:
#z-score normalization
genes_norm = pd.DataFrame(StandardScaler().fit_transform(gs.drop('id', axis=1).to_numpy()))

In [5]:
#for testing accuracy without z-score normalization
#genes_norm = pd.DataFrame(gs.drop('id',axis=1).to_numpy())

In [6]:
#re-insert the ID column
genes_norm.insert(loc=0,column='id',value=genes['id'])

In [1]:
#re-inserting the column names
col_names = [g for g in genes.columns if g not in cols_drop]
genes_norm.columns = col_names

NameError: name 'genes' is not defined

In [9]:
#subset the metadata to extract ID and labels
labels = meta[['id','sample_collection_site']]
#remove samples without labels _ maybe can predict these labels later
labels = labels.dropna()
#inner-join genes and labels frames on ID
d_set = genes_norm.merge(labels, on='id')
#keep categorical labels before one hort encoding eith get_dummies
labels = d_set['sample_collection_site']
#create one-hot encodings of the categorical labels
d_set = pd.get_dummies(d_set, prefix='', columns=['sample_collection_site'])
#keep categorical labels
d_set['sample_collection_site'] = labels
#drop the ID column
d_set = d_set.drop('id', axis=1)

In [10]:
d_set.head()

,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),...,_skin,_small_intestine,_soft_tissue,_spleen,_stomach,_thyroid,_upper_aerodigestive_tract,_urinary_tract,_uvea,sample_collection_site
0,0.588749,-0.202133,1.343858,0.783058,1.014704,-0.333876,-0.419983,-1.154827,1.614989,0.919796,...,0,0,0,0,0,0,0,0,0,lung
1,0.731503,1.492424,0.944880,0.325173,-0.216331,-0.356738,-0.882908,-0.730441,-0.363524,-0.750702,...,0,0,0,0,0,0,0,0,0,central_nervous_system
2,-0.129471,-0.202133,1.366502,-0.060570,0.704678,-0.311458,-0.382578,0.851502,-0.831134,-1.026896,...,1,0,0,0,0,0,0,0,0,skin
3,1.046726,-0.202133,1.018487,0.329713,-0.752319,-0.356738,1.651131,0.564267,-0.130101,-0.119526,...,0,0,0,0,0,0,0,0,0,biliary_tract
4,2.046033,-0.202133,0.064594,0.165990,0.246655,0.282706,2.249149,0.236457,2.159703,0.691694,...,0,0,0,0,0,0,0,1,0,urinary_tract


In [78]:
#finding the tissues from which we have no RNA-seq data
set(list(labels['sample_collection_site'].values)) - set(list(d_set['sample_collection_site'].values))

KeyError: 'sample_collection_site'

ADD THE DISTIBUTIONS AND SPARSITY
SHOULD ALSO DO THE CORRELATION AND OTHER SIMILARITY MATRICES

In [10]:
# split the dataframe into train, test and validation
train = d_set.sample(frac=0.75, random_state=42)
other = d_set.drop(train.index)
test = other.sample(frac=0.4, random_state=42)
val = other.drop(test.index)

#get the titles for label columns to do the x/y split later
label_cols = d_set.iloc[:,-40:].columns

#split the frames into data and label
def xy_split(df):
    #uses label_cols from global
    x = df.drop(label_cols, axis=1)
    return [x, df[label_cols]]

[x_train,y_train],[x_test,y_test],[x_val,y_val] = map(xy_split, [train,val,test])

In [18]:
#pickle the data for later loading for training
for df , name in zip([x_train,x_test,x_val,y_train,y_test,y_val],['x_train.pkl','x_test.pkl','x_val.pkl','y_train.pkl','y_test.pkl','y_val.pkl']):
    df.to_pickle('data/'+name)

In [37]:
# this cell computes the all to all Jaccard similarity
from scipy.stats import entropy
from numpy.linalg import norm
import numpy as np

def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

df = genes.drop('id',axis=1)

jsd_sim = np.zeros(19221*19221).reshape(19221,19221)
for i in tqdm(range(19221)):
    for j in range(i):
        P = df.iloc[:,i]
        Q = df.iloc[:,j]
        jsd_sim[i,j] = JSD(P,Q)

 53%|█████▎    | 10156/19221 [9:03:28<8:05:05,  3.21s/it] 


KeyboardInterrupt: 

In [43]:
#finding the most similar genes
np.where(jsd_sim == max(jsd_sim.reshape(-1)))

(array([2512, 2512, 2581, 3195, 5680, 5853, 5948, 5948, 6106, 6106, 6106,
        6301, 6919, 7394, 7641, 8627, 8707, 8824, 8895, 9151, 9513, 9513,
        9693], dtype=int64),
 array([   1, 1907, 2512, 2512, 2512, 2512, 2512, 3541, 2035, 3022, 3868,
        2512, 3022, 2512, 5853, 2512, 2512, 5948, 1154, 2512, 6925, 8063,
        9513], dtype=int64))